# SKLearn Sample

## Requirements

- Authenticated to gcloud (```gcloud auth application-default login```)

This notebook demonstrate how to deploy iris classifier based on Scikit Learn model using MLP 

In [14]:
!cat requirements.txt

scikit-learn>=1.1.2
# caraml
joblib>=0.13.0,<1.2.0  # >=1.2.0 upon upgrade of kserve's version
cloudpickle==2.0.0

In [15]:
!pip list | grep caraml

caraml-auth-google        0.0.0.post7
caraml-sdk                0.0.0.post73.dev0
caraml-upi-protos         1.0.0

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [6]:
!pip3 install --upgrade -r requirements.txt > /dev/null


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [16]:
# import basic
import warnings
import os
warnings.filterwarnings('ignore')

# Import modeling lib
from sklearn import svm
from sklearn import datasets
from joblib import dump
from sklearn.datasets import load_iris

# Load caraml SDK
import caraml
from caraml.client.caraml_client import CaraMLClient
from caraml.models.model import ModelType


In [17]:
import logging
logging.basicConfig()
# loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
# for logger in loggers:
#     logger.setLevel(logging.DEBUG)

## 1. Initialize CaraML Resources

### 1.1 Set CaraML Server

In [20]:
# Set CARAML Server
CARAML_URL = os.environ.get("CARAML_URL", "http://console.d.ai.golabs.io")
caraml = CaraMLClient(caraml_url=CARAML_URL, use_google_oauth=True)

### 1.2 Set Active Project

`project` represent a project in real life. You may have multiple model within a project.

`caraml.set_project(<project_name>)` will set the active project into the name matched by argument. You can only set it to an existing project. If you would like to create a new project, please do so from the CaraML console at http://localhost:8080/projects/create.

In [21]:
caraml.set_project("sample")

### 1.3 Set Active Model

`model` represents an abstract ML model. Conceptually, `model` in CaraML is similar to a class in programming language. To instantiate a `model` you'll have to create a `model_version`.

Each `model` has a type, currently model type supported by CaraML are: sklearn, xgboost, tensorflow, pytorch, and user defined model (i.e. pyfunc model).

`model_version` represents a snapshot of particular `model` iteration. You'll be able to attach information such as metrics and tag to a given `model_version` as well as deploy it as a model service.

`caraml.get_model(<model_name>, <model_type>)` will get the active model to the name given by parameter, if the model with given name is not found, a new model will be created.

In [22]:
model = caraml.get_or_create_model("sklearn-sample-iris", project_name="sample", model_type=ModelType.SKLEARN)

## 2. Train and Deploy

### 2.1 Train and Upload Model

In [23]:
model_dir = "sklearn-model"
MODEL_FILE = "model.joblib"

clf = svm.SVC(gamma='scale')
iris = datasets.load_iris()
X, y = iris.data, iris.target
clf.fit(X, y)

# Save model to local directory
dump(clf, os.path.join(model_dir, MODEL_FILE))

['sklearn-model/model.joblib']

In [24]:
# Create new version of the model
v = model.new_model_version()
v.start()
v.log_model(model_dir=model_dir)
v.finish()

## 2.2 Deploy Model

In [25]:
endpoint = caraml.deploy(v)

Deploying model sklearn-sample-iris version 3
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:04


Model sklearn-sample-iris version 3 is deployed.
View model version logs: http://console.d.ai.golabs.io/merlin/projects/2/models/361/versions/3/endpoints/897873c6-ca2f-4aa0-8f11-1644cd7ae791/logs


## 2.3 Send Test Request

In [26]:
%%bash -s "$endpoint.url"
curl -v -X POST $1 -d '{
  "instances": [
    [2.8,  1.0,  6.8,  0.4],
    [3.1,  1.4,  4.5,  1.6]
  ]
}'

Note: Unnecessary use of -X or --request, POST is already inferred.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 10.203.170.169:80...
* Connected to sklearn-sample-iris-3.sample.models.id.d.gods.golabs.io (10.203.170.169) port 80 (#0)
> POST /v1/models/sklearn-sample-iris-3:predict HTTP/1.1
> Host: sklearn-sample-iris-3.sample.models.id.d.gods.golabs.io
> User-Agent: curl/8.1.2
> Accept: */*
> Content-Length: 81
> Content-Type: application/x-www-form-urlencoded
> 
} [81 bytes data]
100    81    0     0  100    81      0    340 --:--:-- --:--:-- --:--:--   368< HTTP/1.1 200 OK
< content-length: 21
< content-type: application/json
< date: Mon, 11 Mar 2024 04:45:26 GMT
< server: istio-envoy
< x-envoy-upstream-service-time: 141
< 
{ [21 bytes data]
100   102  100    21  100    81     61    2

{"predictions":[2,2]}

## 3.4 Delete Deployment

In [27]:
caraml.undeploy(v)

Deleting deployment of model sklearn-sample-iris version 3 from enviroment id-dev
